<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/HousePrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Authenticate User </h3>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# importing libraries
import pandas as pd


<h3>FEtching data from clo9ud storage </h3>

In [0]:
#!gsutil cp gs://mm_tensorflow_bucket/pp-2018-part2.csv	 /tmp/pp-2018-part2.csv
#!gsutil cp gs://mm_tensorflow_bucket/pp-2015-part1.csv	 /tmp/pp-2015-part1.csv

# Get gsc service
def get_gcs_service():
  from googleapiclient.discovery import build
  gcs_service = build('storage', 'v1')
  return gcs_service

def download_from_storage(bucketname, bucketfilename, tmpfile):
  from apiclient.http import MediaIoBaseDownload
  gcs_service = get_gcs_service()
  with open(tmpfile, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=bucketname,
                                              object=bucketfilename)
    media = MediaIoBaseDownload(f, request)

    done = False
    while not done:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      _, done = media.next_chunk()

  print('Download complete')


  

<h3> Readign the dataset from Land Registry for Sold Prices</h3>

In [0]:
def read_land_registry_datasets(filename):
  
  df =  pd.read_csv(filename, names=['unique_id',	'price_paid','deed_date','postcode',
                                                       'property_type',	'new_build',	'estate_type',	
                                                       'saon',	'paon',	'street','locality',
                                                       'town',	'district',	'county',	'transaction_category','linked_data_uri']).fillna('') 
  print ('Found {} in {}'.format(df.shape[0], filename))
  return df

In [0]:
from datetime import datetime
def find_sales_for_postcode(postcode, filenames):
  print('Finding sold prices for postcode:{}'.format(postcode))
  lr_datasets = [read_land_registry_datasets(filename) for filename in filenames]
  
  results = []
  for df in lr_datasets:
    restricted = df[df['postcode'].str.contains('BN2 5')][['price_paid', 'deed_date', 'postcode', 'property_type', 'saon', 'paon', 'street']]
    flats_only = restricted[restricted['property_type'] == 'F']
    print('found {} flats for postcode {} '.format(flats_only.shape[0], postcode))
    results.append(flats_only)
    
  all_res =  pd.concat(results)
  print('All res has shape{}'.format(all_res.shape))
  return all_res

                                        

In [0]:
filenames = ['/tmp/pp-2018-part2.csv', '/tmp/pp-2015-part1.csv']
final = find_sales_for_postcode('BN2 5', filenames)
print('Exportign to csv...with {}'.format(final.shape))
filename = '/tmp/Results.csv'.format(datetime.now().strftime('%Y%m%d-%H:%M'))
final.to_csv(filename, header=True)


Finding sold prices for postcode:BN2 5
Found 511575 in /tmp/pp-2018-part2.csv
Found 504743 in /tmp/pp-2015-part1.csv
found 32 flats for postcode BN2 5 
found 50 flats for postcode BN2 5 
All res has shape(82, 7)
Exportign to csv...with (82, 7)


In [0]:

print('Copying to bucket')
!gsutil cp /tmp/Results.csv gs://mm_tensorflow_bucket/                                       


Copying to bucket
Copying file:///tmp/Results.csv [Content-Type=text/csv]...
/ [1 files][  5.7 KiB/  5.7 KiB]                                                
Operation completed over 1 objects/5.7 KiB.                                      


<h3> Private REntal Market </h3>

In [0]:
filename = "/tmp/PrivateRentalMarket_Apr18_Mar19.csv"
download_from_storage("mm_tensorflow_bucket", "PrivateRentalMarket/PrivateRentalMarket_Apr18_Mar19.csv",   filename)

Download complete


In [0]:
!tail /tmp/PrivateRentalMarket_Apr18_Mar19.csv


1615,E07000080,Forest of Dean,240,581,550,575,600
1620,E07000081,Gloucester,650,638,595,650,675
1625,E07000082,Stroud,490,675,625,675,720
1630,E07000083,Tewkesbury,300,655,585,650,700
NA,E10000027,Somerset,2980,623,575,625,665
3305,E07000187,Mendip,520,646,595,635,695
3310,E07000188,Sedgemoor,610,611,550,600,650
3325,E07000189,South Somerset,970,617,570,625,650
3315,E07000190,Taunton Deane,660,631,575,625,675
3320,E07000191,West Somerset,220,606,550,600,650


In [0]:
def read_file(file_name, city):
  print('Filtering rents for :{}'.format(city))
  df = pd.read_csv(file_name, header=0, encoding = "ISO-8859-1") 
  df.rename(columns={'LA_Code': 'LocaAreaCode', 'AREA_CODE' : 'Area_Code',
                     'CNT' : 'Total_Rent_count', 'AVG':'Average_Rent', 'LQ': 'Lower_Quartile_Rent',
                     'MED' : 'Median_Rent', 'UQ' : 'Upper_Quartile_Rent'}, inplace=True)
  return df[df['Area'].str.contains(city)]

In [0]:
data_set = read_file('/tmp/PrivateRentalMarket_Apr18_Mar19.csv', 'Tower')
data_set

Filtering rents for :Tower


,LocaAreaCode,Area_Code,Area,Total_Rent_count,Average_Rent,Lower_Quartile_Rent,Median_Rent,Upper_Quartile_Rent
231,5900.0,E09000030,Tower Hamlets,1090,1853,1562,1733,1950
